## Einlesen aller Super-League-Spiele seit 2003 (seit Einführung des aktuellen Modus)

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
from datetime import timedelta

## Alle Spiele einlesen
Auf "transfermarkt.com" mache ich Requests von allen Teams, die in den vergangenen 15 Jahren in der Super League spielten. Mit Beautifulsoup hole ich alle Daten heraus: Teams, Resultat, Zuschauer, Spielzeit, etc.

In [ ]:
total_spiele = []


vereinsliste = [["neuchatel-xamax-fcs/spielplan/verein/9084","Xamax"],
                ["neuchatel-xamax/spielplan/verein/625","Xamax"], 
                ["fc-luzern/spielplan/verein/434","Luzern"],
                ["fc-st-gallen-1879/spielplan/verein/257","St.Gallen"],
                ["fc-thun/spielplan/verein/938","Thun"],
                ["grasshopper-club-zurich/spielplan/verein/504","Grasshoppers"],
                ["fc-vaduz/spielplan/verein/163","Vaduz"],
                ["fc-basel-1893/spielplan/verein/26","Basel"],
                ["fc-zurich/spielplan/verein/260","Zürich"],
                ["bsc-young-boys/spielplan/verein/452","Young Boys"],
                ["fc-lausanne-sport/spielplan/verein/527","Lausanne"],
                ["servette-fc/spielplan/verein/61","Servette"],
                ["fc-sion/spielplan/verein/321","Sion"],
                ["ac-bellinzona/spielplan/verein/2047","Bellinzona"], 
                ["fc-aarau/spielplan/verein/116","Aarau"],
                ["fc-lugano/spielplan/verein/2790","Lugano"], 
                ["fc-schaffhausen/spielplan/verein/1277","Schaffhausen"],
                ["yverdon-sport-fc/spielplan/verein/322","Yverdon"],
                ["fc-wil-1900/spielplan/verein/850","Wil"]]


for verein in vereinsliste:
    
    jahre = list(range(2003,2018))
    for jahr in jahre:
        soup = ""
        spielliste = []
        wettbewerb_art = []
        allespiele = ""
        
        url = "https://www.transfermarkt.ch/" + verein [0]+ "/plus/0?saison_id="+str(jahr)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        
        r = requests.get(url, headers=headers)
        print(url)
        

        soup = BeautifulSoup(r.text, 'html.parser')

        wettbewerb_art = soup.find_all('div', {'class':'box'})

        # Schlaufe, um alle Barragespiele auszuschliessen und nur die "regulären" Super-League-Spiele zu erhalten.
        for i in range (0,len(wettbewerb_art)):
            if "Super League" in wettbewerb_art[i].text and "Spielsystem" in wettbewerb_art[i].text and "Barrage" not in wettbewerb_art[i].text:
                allespiele = wettbewerb_art[i].text
                print(i)



        spielliste = allespiele.split("\t\t\t\t\t\t\t\t\n\n")
        
        for i in range(1,len(spielliste)): 
            spielliste[i] = spielliste[i].replace("So. ","Sun")
            spielliste[i] = spielliste[i].replace("Mo. ","Mon")
            spielliste[i] = spielliste[i].replace("Di. ","Tue")
            spielliste[i] = spielliste[i].replace("Mi. ","Wed")
            spielliste[i] = spielliste[i].replace("Do. ","Thu")
            spielliste[i] = spielliste[i].replace("Fr. ","Fri")
            spielliste[i] = spielliste[i].replace("Sa. ","Sat")
            spielliste[i] = spielliste[i].replace("\xa0\xa0"," ")

        
        #Schlaufe, um die einzelnen Werte (Teams, Resultat, etc.) herauszufiltern
        
        runde = 0
        for spiel in spielliste:
            if ".)" in spiel:
                spieldaten = spiel.split("\n")
                tag = spieldaten[0][0:3]
                datum = spieldaten[0].replace(" ","")[3:14]
                zeit = spieldaten[1]
                spielzeit = pd.to_datetime(tag + " " + datum + " " + zeit, format='%a %d.%m.%Y %H:%M')
                heim = spieldaten[2]
                if heim == "H":
                    heimteam = verein[1]
                    austeam = spieldaten[5][:-5]
                    heimrang = spieldaten[3][1:2]
                    ausrang = spieldaten[5][-3:-2]
                elif heim == "A":
                    heimteam = spieldaten[5][:-5]
                    austeam = verein[1]
                    heimrang = spieldaten[5][-3:-2]
                    ausrang = spieldaten[3][1:2]

                zuschauer = spieldaten[7].replace(".","")
                tore = spieldaten[8].replace(" ","").split(":")
                tore_heim = tore[0]
                tore_aus = tore[1]
                saison = str(jahr)+"/"+str(jahr+1)
                runde = runde + 1

                mini_dict = {"Saison":saison,
                             "Runde":runde,
                            "Tag":tag,
                            "Datum": datum,
                            "Zeit" : zeit,
                            "Spielzeit":spielzeit,
                            "Heimteam": heimteam,
                            "Rang Heimteam":heimrang,
                            "Auswärtsteam": austeam,
                            "Rang Auswärtsteam":ausrang,
                            "zuschauer":zuschauer,
                            "tore_heim":tore_heim,
                            "tore_aus":tore_aus}

                total_spiele.append(mini_dict)
       


## Spiel-Dataframe erstellen und speichern

In [ ]:
df = pd.DataFrame(total_spiele)

In [ ]:
df.to_csv('allespiele.csv')

In [ ]:
df = pd.read_csv("allespiele.csv")
#pd.options.display.max_rows = 6000

Da ich alle Spiele von jedem Team erfasst habe, sind die Spiele nun doppelt vorhanden. Dies werde ich ändern, muss dafür zunächst aber alle Teamnamen auf eine Schreibweise definieren:

In [ ]:
def changename(elem):
    
    elem = elem.lower()
    if 'basel' in elem:
        return 'Basel'
    if 'hoppers' in elem:
        return 'Grasshoppers'
    if 'wil' in elem:
        return 'Wil'
    if 'young' in elem:
        return 'YB'
    if 'st.gallen' in elem:
        return 'St.Gallen'
    if 'st. gallen' in elem:
        return 'St.Gallen'
    if 'zürich' in elem:
        return 'Zürich'
    if 'sion' in elem:
        return 'Sion'
    if 'servette' in elem:
        return 'Servette'
    if 'luzern' in elem:
        return 'Luzern'
    if 'thun' in elem:
        return 'Thun'
    if 'basel' in elem:
        return 'Basel'
    if 'aarau' in elem:
        return 'Aarau'
    if 'yverdon' in elem:
        return 'Yverdon'
    if 'xamax' in elem:
        return 'Xamax'
    if 'lausanne' in elem:
        return 'Lausanne'
    if 'bellinzona' in elem:
        return 'Bellinzona'
    if 'vaduz' in elem:
        return 'Vaduz'
    if 'lugano' in elem:
        return 'Lugano'
    if 'schaffhausen' in elem:
        return 'Schaffhausen'
    

In [ ]:
df['Heimteam'] = df['Heimteam'].apply(changename)
df['Auswärtsteam'] = df['Auswärtsteam'].apply(changename)

Nun lasse ich die Duplikate (die doppelt erfassten Spiele) verschwinden

In [ ]:
df = df.drop_duplicates(subset=['Heimteam', 'Auswärtsteam',"Saison","Spielzeit"], keep="first")

Und speichere das ganze geputzte Dataframe

In [ ]:
df.to_csv('allespiele_clean.csv')

In [3]:
df = pd.read_csv("allespiele_clean.csv")

Um die Hinrunden- bzw. Rückrundenpunkte der Teams herauszufinden, muss ich zunächst berechnen, wieviele Punkte die Teams in jedem Spiel geholt haben. Das ist im df bisher nicht erfasst, sondern nur die Tore.

In [13]:
df["punkteaus"] = 0
df["punkteheim"] = 0
df['punkteaus'] = np.where((df['tore_aus'] > df['tore_heim']), df['punkteaus'] + 3, df["punkteaus"])
df['punkteheim'] = np.where((df['tore_heim'] > df['tore_aus']), df['punkteheim'] + 3, df["punkteheim"])
df['punkteaus'] = np.where((df['tore_aus'] == df['tore_heim']), df['punkteaus'] + 1, df["punkteaus"])
df['punkteheim'] = np.where((df['tore_aus'] == df['tore_heim']), df['punkteheim'] + 1, df["punkteheim"])

In [14]:
# Liste der Vereine erstellen, um später durch die Vereinsliste iterieren zu können
teamliste = set(list(df['Heimteam']))

In [15]:
#dito mit den Saisons
alle_saisons = set(list(df['Saison']))

Nun erstelle ich für jedes der Super-League-Teams der vergangenen 15 Jahre eine df-Spalte,... 

In [16]:
for team in teamliste:
    df[team] = 0

...um darin nun die erreichten Punkte pro Spiel einsetzen zu können

In [17]:
for team in teamliste:
    df[team] = np.where((df['Heimteam'] == team), df[team] + df['punkteheim'], df[team])
    df[team] = np.where((df['Auswärtsteam'] == team), df[team] + df['punkteaus'], df[team])
  

Auch dieses abgeänderte Dataframe speichere ich ab, nachdem ich die Spiele zeitlich geordnet habe

In [ ]:
df = df.sort_values('Spielzeit')
df.to_csv('allespiele_punkte.csv')
